<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/%E3%80%90%E8%A9%95%E4%BE%A1%E7%94%A8%E3%80%91TrainingSet%E3%81%AE%E3%83%AA%E3%82%B9%E3%83%88CSV%E3%81%A8%E7%95%AA%E5%8F%B7%E4%BB%98%E3%81%8D%E7%94%BB%E5%83%8F%E3%82%92%E4%BD%9C%E6%88%90_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**【評価用】TrainingSetのリストCSVを作成**


In [1]:
"""
folder-------grav 1000 images
       |-----cont 1000 images

 ↓ ランダムにピックアップ

test_folder----pre---treated (=grav) 50 images
            |      |-untreated (=cont) 50 images
            |--post--treated (=grav) 50 images
                   |-untreated (=cont) 50 images
※pre:指導前、post:指導後
→CSVでリストを作成（それぞれ画像0-99、pre&postで対応表作成）
"""

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os
import pandas as pd
import glob
import random
import shutil
import pandas as pd
random.seed(314)

grav_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_640px"  #test画像(classごとにフォルダ分けあり)のパス
cont_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/untreated_640px"  #test画像(classごとにフォルダ分けあり)のパス
csv_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp"   #画像のリストをCSVに書き出した保存先
img_save_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp/images"   #通し番号を記入した画像を保存するパス
pdf_save_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp"   #画像を番号順にpdf化したものを保存するパス
zip_save_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp"   #評価画像と回答用csvの組み合わせを圧縮して保存
reply_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp/reply"   #返信のcsvを保存するフォルダ


for dir in [csv_dir, img_save_dir, pdf_save_dir, zip_save_dir, reply_dir]:
    if os.path.exists(dir) == False:
        os.makedirs(dir)

for phase in [0,1,2]:
    os.makedirs(f"{img_save_dir}/phase_{phase}", exist_ok=True)


In [10]:
shutil.rmtree("/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp/images")

#**新たに評価用CSVを作成**

In [12]:
def split_list(lst):
    random.shuffle(lst)
    return lst[:len(lst)//3], lst[len(lst)//3:2*len(lst)//3], lst[2*len(lst)//3:]

grav = random.sample(glob.glob(f"{grav_dir}/*"), 99)    
grav_0, grav_1, grav_2 = split_list(grav)

cont = random.sample(glob.glob(f"{cont_dir}/*"), 99)    
cont_0, cont_1, cont_2 = split_list(cont)


phase_0 = grav_0 + cont_0
phase_1 = grav_1 + cont_1
phase_2 = grav_2 + cont_2
random.shuffle(phase_0)
random.shuffle(phase_1)
random.shuffle(phase_2)

In [13]:
def list_for_df(path_list, phase):
    idxs, labels = [],[]
    for idx, path in enumerate(path_list):
        idxs.append(f"phase{phase}_{str(idx).zfill(2)}")
        if os.path.basename(os.path.dirname(path)).split("_")[0] == "treated":
              labels.append(1) # grav 
        elif os.path.basename(os.path.dirname(path)).split("_")[0] == "untreated":
              labels.append(0) # cont
    return path_list, idxs, labels

paths_0, idxs_0, labels_0 = list_for_df(phase_0, 0)
paths_1, idxs_1, labels_1 = list_for_df(phase_1, 1)
paths_2, idxs_2, labels_2 = list_for_df(phase_2, 2)
idxs = idxs_0 + idxs_1 + idxs_2
labels = labels_0 + labels_1 + labels_2
paths = paths_0 + paths_1 + paths_2

print(len(idxs), len(labels), len(paths))



198 198 198


In [14]:
columns = ["idx", "label", "path"]
participants = ["ishihara", "ohagi", "kinoshita", "takamine", "hayashi", "fukuyama", "makino", "motomura", "iwasaki", "komatsu", "koizumi"] 
columns.extend(participants)

df = pd.DataFrame(index=[], columns=columns)
df["idx"] = idxs
df["label"] = labels
df["path"] = paths

In [15]:
#DataFrameをセーブ
df.to_csv(f"{csv_dir}/hum_pred.csv")

In [ ]:
#選択した画像をフォルダに移動
for i, (idx, path) in enumerate(zip(df["idx"],df["path"])):
    print(idx,path)
    shutil.copy(path, f"{img_save_dir}/phase_{idx[5]}/{idx}.jpg") #idx[5]-->左から5文字目


In [17]:
import zipfile
import os

def zip_directory(directory_path, zip_file_path, password):
    with zipfile.ZipFile(zip_file_path, 'w') as zip_file:
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, os.path.relpath(file_path, directory_path), compress_type=zipfile.ZIP_DEFLATED)
    zip_file.setpassword(password.encode('utf-8'))

%cd /content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp/images
for dir in os.listdir():
    zip_directory(dir, f"{dir}.zip", 'bootcamp')

/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp/images


In [ ]:
df

,idx,label,path,ishihara,ohagi,kinoshita,takamine,hayashi,fukuyama,makino,motomura,iwasaki,komatsu,koizumi
0,phase0_00,0,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,phase0_01,0,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,phase0_02,1,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,phase0_03,0,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,phase0_04,0,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,phase2_61,1,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,phase2_62,0,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,phase2_63,1,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,phase2_64,1,/content/drive/MyDrive/Deep_learning/Olympia_d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#**評価用画像を作成**
通し番号を合成して新しいフォルダに保存

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import shutil

# save_path = img_save_dir

# #save_pathがあれば削除して新しく作り直す
# try:
#     shutil.rmtree(save_path)
#     os.makedirs(save_path)
# except FileNotFoundError:
#     os.makedirs(save_path)
#     pass

# #CSVに対応する画像のパスを取得

# img_path_list = []
# for i in range(len(df3)):
#     img_name = df3.iloc[i,0]
#     img_class = df3.iloc[i,1]
#     img_path_list.append(test_dir + "/" +img_class+ "/" + img_name)

# print(img_path_list)

def write_text(image_path, text, save_path):
    img = cv2.imread(image_path)
    # 文字を加える                                                                                      
    cv2.putText(img, text, (20, 60),
               cv2.FONT_ITALIC, 1.5,
               (255,255,255), 4, cv2.LINE_AA)
    #cv2.imwrite(save_path +"/" +os.path.basename(image_path), img)
    cv2.imwrite(save_path, img)
    return(img)

for phase in [0,1,2]:
    dir = f"/content/drive/MyDrive/Deep_learning/Olympia_dataset/grav_bootcamp/images/phase_{phase}"
    for idx, path in enumerate(glob.glob(f"{dir}/*")):
        im = write_text(path, str(os.path.basename(path).split(".")[0]), path)
        cv2_imshow(im)



#**画像を連結して印刷用のPDFファイルにする**

In [25]:
!pip install img2pdf --q
import os
import img2pdf
from PIL import Image # img2pdfと一緒にインストールされたPillowを使います

for phase in [0,1,2]:
    pdfFileName = f"phase_{phase}.pdf"
    path = f"{img_save_dir}/phase_{phase}/*"
    ext = ".jpg"
 
    with open(pdfFileName, "wb") as files:
      files.write(img2pdf.convert([i for i in glob.glob(path) if i.endswith(ext)]))


In [ ]:
##PDFにパスワードをかける

#input
!pip install PyPDF4
from PyPDF4 import PdfFileReader
from PyPDF4 import PdfFileWriter

for phase in [0,1,2]:
    # パスワード設定用PDF
    pdf = f"phase_{phase}.pdf"

    # パスワード設定用PDFをバイナリモードで開く
    pdf_open = open(pdf,"rb")

    # PdfFileReaderオブジェクト生成
    pdf_reader = PdfFileReader(pdf_open)

    # PdfFileWriterオブジェクト
    pdf_writer = PdfFileWriter()

    # PDFのページ数取得
    num = pdf_reader.numPages

    # PDFページコピー
    for cp in range(num):
        page = pdf_reader.getPage(cp)
        pdf_writer.addPage(page)

    # パスワードの指定
    password = "6903"
    pdf_writer.encrypt(password)

    # 暗号化情報を書き込み
    with open(pdf.replace('.pdf','') + r'_encrypt.pdf','wb') as f:
        pdf_writer.write(f)

#**評価用CSVと画像pdfをまとめてzip化する**

In [ ]:
import zipfile

with zipfile.ZipFile('evaluation.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    new_zip.write('hum_eval.csv', arcname='hum_eval.csv')
    new_zip.write('images.pdf', arcname='750px_images.pdf')

In [ ]:
with zipfile.ZipFile(zip_save_dir+"/evaluation.zip", 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    new_zip.write('hum_eval.csv', arcname='hum_eval.csv')
    new_zip.write('images.pdf', arcname='750px_images.pdf')